# Set the environment

In [1]:
# set the envoiroment
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import flux_variability_analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import escher
from escher import Builder 
from utils import show_map
from utils.check_precursor_problem import check_precursor_problem
map_loc = "../data/fermentation map for Figure S1.json" #Import the map that has been drawn using escher.
map_loc1 = "../data/adding all the metabolites exchange reaction.json"

## load the model

In [2]:
model = cobra.io.load_json_model("../models/updated_model.json")# Import the model.
model.solver = 'glpk'

## positive control 
   - Simulation of Table 2. number 1. 
   - The strain can grow. 

In [3]:
M = model.copy()
M.objective = M.reactions.Biomass
sol = M.optimize()
print ("When making the biomass production reaction as the objective reaction, the modelled biomass of this organism is "+ str(sol.fluxes.loc['Biomass']) +".")

When making the biomass production reaction as the objective reaction, the modelled biomass of this organism is 0.06334222910440473.


In [4]:
model.reactions.get_by_id("NGAM").bounds = (0,1000)

## Blocking the reaction for respiration and do FBA

### Blocking the reaction for oxygen importing. 
   - Simulation of Table 2. number 2.
   - The strain cannot grow. 

In [5]:
M = model.copy()
M.reactions.get_by_id("T_flux_oxygen").bounds = (0,0) # Block this reaction.
M.objective = M.reactions.Biomass  
sol = M.optimize()
print ("When we block the reaction for respiration, the modelled biomass of this organism is "+ str(sol.fluxes.loc['Biomass']) +".")

When we block the reaction for respiration, the modelled biomass of this organism is 0.0.


### Use the lactic acid export reaction as objective reaction
- Simulation of Table 2. number 3. 

In [6]:
M= model.copy()
M.reactions.get_by_id("T_flux_oxygen").bounds = (0,1000)
M.objective = M.reactions.get_by_id('T_flux_(S)__lactic_acid')
sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index

print ("The flux for lactic acid production is", sol.fluxes.loc['T_flux_(S)__lactic_acid'],".")

The flux for lactic acid production is 0.0 .


### Use the ethanol export reaction as objective reaction
- Simulation of Table 2 number 4.

In [7]:
M= model.copy()
M.reactions.get_by_id('T_flux_ethanol').bounds = (-1000,1000)
M.reactions.get_by_id("T_flux_oxygen").bounds = (0,1000)
M.objective = M.reactions.get_by_id('T_flux_ethanol')
sol = M.optimize()
print ("The flux for ethnol production is" ,sol.fluxes.loc['T_flux_ethanol'],".")

The flux for ethnol production is 0.0 .


### And pyruvate decarboxylase reaction and then use the pyruvate decarboxylase reaction as objective reaction
- Simulation of Table 2. number 5.

In [8]:
# And pyruvate decarboxylase reaction and then use the pyruvate decarboxylase reaction as objective reaction
M= model.copy()
reaction = Reaction('pyruvate_decarboxylase_to_acetaldehyde')
reaction.name = 'pyruvate decarboxylase _to_acetaldehyde reaction'
reaction.lower_bound = 0.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"H+": -1.0,
                         "pyruvate": -1.0,
                         "acetaldehyde": 1.0,
                         "CO2": 1.0})

M.reactions.get_by_id("T_flux_oxygen").bounds = (0,1000)
M.objective = M.reactions.get_by_id('T_flux_ethanol')
sol = M.optimize()

print ("The flux through ethanol is", sol.fluxes.loc['T_flux_ethanol'],".")

The flux through ethanol is 0.0 .


###  Making protonation reaction of lactate to lactic acid as reversible reaction.
- Simulation of Table 2 number 6 and Figure S1.

In [9]:
M= model.copy()
M.reactions.get_by_id("prot_prot__lactate").bounds = (-1000,1000)
M.objective = M.reactions.get_by_id('T_flux_(S)__lactic_acid')
sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index

print ("The flux for lactic acid production is", sol.fluxes.loc['T_flux_(S)__lactic_acid'],".")
b = show_map(sol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

The flux for lactic acid production is 2.000000000000001 .


### Making all the protonation reactions in the genome wide map reversible and checking the production of ethanol

In [10]:
M= model.copy()
for rxn in M.reactions:
    if "prot_prot__" in rxn.id:
        rxn.bounds = (0,0)
M.reactions.get_by_id("T_flux_oxygen").bounds = (0,1000)
M.objective = M.reactions.get_by_id ('T_flux_ethanol')
sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index

print ("The flux for ethnol production is", sol.fluxes.loc["T_flux_ethanol"])

The flux for ethnol production is 0.0


###  Use the ethanol export reaction as reversible and check the production of ethanol.
- Simulation of Table 1 number 7. 
- After running FBA the flux is 0. The reason for this is NADH inbalance. From glucose to pyruvate, it will produce two molecules of NADH, or NADPH, however, in order to produce ethnol 4 molecules of NADH or NADPH are requred. A possible solution is to add the pyruvate exchange reaction. 

In [11]:
M= model.copy()
M.reactions.get_by_id("T_ABC__transporter_ethanol").bounds = (-1000,1000)

for rxn in M.reactions:
    if "prot_prot__" in rxn.id:
        rxn.bounds = (0,0)
M.reactions.get_by_id("T_flux_oxygen").bounds = (0,1000)
M.objective = M.reactions.get_by_id ('T_flux_ethanol')
sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index
# for rxn in l:
#     print (rxn,M.reactions.get_by_id(rxn).reaction,sol.fluxes.loc[rxn])
print ("The flux for ethnol production is", sol.fluxes.loc["T_flux_ethanol"])

The flux for ethnol production is 0.0


### And pyruvate decarboxylase reaction and then use the pyruvate decarboxylase reaction as objective reaction
- Simulation of Table 2 number 8.

In [12]:
# And pyruvate decarboxylase reaction and then use the pyruvate decarboxylase reaction as objective reaction
M= model.copy()
M.reactions.get_by_id("T_ABC__transporter_ethanol").bounds = (-1000,1000)
reaction = Reaction('pyruvate_decarboxylase')
reaction.name = 'pyruvate decarboxylase reaction'
reaction.lower_bound = 0.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"H+": -1.0,
                         "pyruvate": -1.0,
                         "acetaldehyde": 1.0,
                         "CO2": 1.0})
M.reactions.get_by_id("T_flux_oxygen").bounds = (0,1000)
M.objective = M.reactions.get_by_id('T_flux_ethanol')
sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index
print ("The flux through ethanol is", sol.fluxes.loc['T_flux_ethanol'],".")

The flux through ethanol is 1.9999999999999996 .


### And a _ethanol diffusion reaction and  pyruvate decarboxylase reaction and then use the ethanol export reaction as objective reaction
- Simulation of Table 2 number 9.

In [13]:
M= model.copy()

reaction = Reaction('pyruvate_decarboxylase') 
reaction.name = 'pyruvate decarboxylase reaction'
reaction.lower_bound = 0.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"H+": -1.0,
                         "pyruvate": -1.0,
                         "acetaldehyde": 1.0,
                         "CO2": 1.0})

reaction = Reaction('passive_ethanol_transport')
reaction.name = 'passive ethanol transport'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"ethanol_ex": -1.0,
                         "ethanol": 1.0})

M.reactions.get_by_id("T_flux_oxygen").bounds = (0,1000)
M.objective = M.reactions.get_by_id('T_flux_ethanol')
sol = M.optimize()

print ("The flux through pyruvate_decarboxylase is,", sol.fluxes.loc["T_flux_ethanol"])

The flux through pyruvate_decarboxylase is, 2.0000000000000004


### Use lactate dehydrogenase, grow anaerobically?
- Simulation of Table 2 number 10.

In [14]:
M= model.copy()
M.reactions.get_by_id("prot_prot__lactate").bounds = (-1000,1000)
M.reactions.get_by_id("T_flux_oxygen").bounds = (0,1000)

M.objective = M.reactions.get_by_id('Biomass')
sol = M.optimize()
print ("The flux through biomass reaction is", sol.fluxes.loc['Biomass'],".")


The flux through biomass reaction is 0.0 .


### Use fermentation to ethanol, grow anaerobically?
- Simulation of Table 2 number 11.

In [15]:
M= model.copy()
reaction = Reaction('pyruvate_decarboxylase') 
reaction.name = 'pyruvate decarboxylase reaction'
reaction.lower_bound = 0.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"H+": -1.0,
                         "pyruvate": -1.0,
                         "acetaldehyde": 1.0,
                         "CO2": 1.0})

reaction = Reaction('passive_ethanol_transport')
reaction.name = 'passive ethanol transport'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"ethanol_ex": -1.0,
                         "ethanol": 1.0})

M.reactions.get_by_id("T_flux_oxygen").bounds = (0,1000)
M.objective = M.reactions.get_by_id('Biomass')
sol = M.optimize()

print ("The flux through biomass reaction is", sol.fluxes.loc["Biomass"])

The flux through biomass reaction is 0.0


## Check ATP production from the proposed fermentation pathway
- Simulation of Table 1 number 12.

### 4 protons can be pumped out, 2 ATP will be produced.

In [16]:
def different_protons_pumped_out (s):    
    M= model.copy()
    Psi = Metabolite(
        'Psi',
        formula='',
        name='PSI is a unit of pressure expressed in pounds of force per square inch of area',
        compartment='cytosol')
    M.reactions.get_by_id('T_symporter_(S)__lactic_acid').remove_from_model()
    reaction = Reaction('T_symporter_(S)__lactic_acid')
    reaction.name = 'T_symporter_(S)__lactic_acid'
    reaction.lower_bound = -1000.  # This is the default
    reaction.upper_bound = 1000.  # This is the default
    M.add_reactions([reaction])
    reaction.add_metabolites({"(S)__lactic_acid":-1.0,
                             "H+":-s,
                             "(S)__lactic_acid_ex":1.0,
                               Psi: s,
                              "H+_ex":s})

    M.reactions.get_by_id("prot_prot__lactate").bounds = (-1000,1000)

    M.reactions.get_by_id("T_flux_oxygen").bounds = (0,0)
    M.reactions.NGAM.bounds = (0,1000)

    M.objective = M.reactions.get_by_id('NGAM')
    sol = M.optimize()
    return(sol.fluxes.loc['NGAM'])

In [17]:
df_protons_pumped_out = pd.DataFrame(columns=['protons', 'ATP'])
df_protons_pumped_out["protons"]=np.arange(0, 8.1, step=2)
df_protons_pumped_out = df_protons_pumped_out.set_index(df_protons_pumped_out["protons"],)
for n in df_protons_pumped_out.index:
    df_protons_pumped_out["ATP"].loc[n] = different_protons_pumped_out(n)
df_protons_pumped_out[["ATP"]]

C:\Users\yzhang2\Anaconda3\envs\archeapy36\lib\site-packages\pandas\core\indexing.py:205 SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,ATP
protons,
0.0,0
2.0,1
4.0,2
6.0,3
8.0,4


## Check the biomass production

In [18]:
M.objective = M.reactions.Biomass
sol = M.optimize()
print (sol )

<Solution 0.000 at 0x27f31b8dd30>


### Check the precusor which cannot be produced

In [19]:
check_precursor_problem(M, ["Biomass"])

{'2,3__bis__O__phytanyl__sn__glycerol',
 'AMP',
 'CMP',
 'FAD',
 'GMP',
 'L__arginine',
 'L__asparagine',
 'L__aspartate',
 'L__cysteine',
 'L__glutamate',
 'L__glutamine',
 'L__histidine',
 'L__homocysteine',
 'L__isoleucine',
 'L__leucine',
 'L__lysine',
 'L__methionine',
 'L__phenylalanine',
 'L__proline',
 'L__serine',
 'L__threonine',
 'L__tryptophan',
 'L__tyrosine',
 'NAD+',
 'NADH',
 'NADP+',
 'NADPH',
 'UDP__alpha__N__acetyl__D__glucosamine',
 'UMP',
 'coenzyme_A',
 'coenzyme_B12',
 'dAMP',
 'dCMP',
 'dGMP',
 'dTMP',
 'glycine',
 'glycogen',
 'norspermidine',
 'norspermine',
 'phosphoenolpyruvate',
 'propanoyl__CoA',
 'putrescine',
 "pyridoxal_5'__phosphate",
 'representative_quinone',
 'saturated_calditolcaldarchaeol',
 'saturated_glycolipidcaldarchaeol',
 'saturated_inositolcaldarchaeol',
 'saturated_inositolcalditolcaldarchaeol',
 'saturated_inositolglycolipidcaldarchaeol',
 'saturated_sulfoglycolipidcaldarchaeol',
 'shikimate',
 'sn__glycerol__3__phosphate',
 'spermidine',

### Add the precursors and check the biomass production

In [20]:
reaction = Reaction('coenzyme_B12_flux')
reaction.name = 'coenzyme_B12_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"coenzyme_B12": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('norspermidine_flux')
reaction.name = 'norspermidine_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"norspermidine": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('spermidine_flux')
reaction.name = 'spermidine_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"spermidine": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('thiamin_diphosphate_flux')
reaction.name = 'thiamin_diphosphate_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"thiamin_diphosphate": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('representative_quinone_flux')
reaction.name = 'representative_quinone_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"representative_quinone": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('norspermine_flux')
reaction.name = 'norspermine_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"norspermine": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
M.objective = M.reactions.Biomass
M.optimize()

,fluxes,reduced_costs
Biomass,0.0,1.110223e-16
Biomass__fucose,0.0,-1.997507e+00
Biomass__phenol,0.0,-2.014550e+00
Biomass__casamino__acids,0.0,-1.992909e+00
GAM,0.0,0.000000e+00
...,...,...
norspermidine_flux,0.0,0.000000e+00
spermidine_flux,0.0,0.000000e+00
thiamin_diphosphate_flux,0.0,6.938894e-18
representative_quinone_flux,0.0,0.000000e+00


## Check biomass from the proposed fermentation pathway with adding the procusors

### 0 protons can be pumped out, biomas will not be produced.

In [21]:
M= model.copy()
Psi = Metabolite(
    'Psi',
    formula='',
    name='PSI is a unit of pressure expressed in pounds of force per square inch of area',
    compartment='cytosol')
M.reactions.get_by_id('T_symporter_(S)__lactic_acid').remove_from_model()
reaction = Reaction('T_symporter_(S)__lactic_acid')
reaction.name = 'T_symporter_(S)__lactic_acid'
reaction.lower_bound = -1000.  # This is the default
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"(S)__lactic_acid":-1.0,
                         "H+":-0.0,
                         "(S)__lactic_acid_ex":1.0,
                           Psi: 0.0,
                          "H+_ex":0.0})

M.reactions.get_by_id("prot_prot__lactate").bounds = (-1000,1000)

# addubg the precusors 
M.reactions.get_by_id("T_flux_oxygen").bounds = (0,0)
M.reactions.NGAM.bounds = (0,1000)
reaction = Reaction('coenzyme_B12_flux')
reaction.name = 'coenzyme_B12_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"coenzyme_B12": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('norspermidine_flux')
reaction.name = 'norspermidine_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"norspermidine": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('spermidine_flux')
reaction.name = 'spermidine_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"spermidine": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('thiamin_diphosphate_flux')
reaction.name = 'thiamin_diphosphate_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"thiamin_diphosphate": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('representative_quinone_flux')
reaction.name = 'representative_quinone_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"representative_quinone": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('norspermine_flux')
reaction.name = 'norspermine_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"norspermine": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
M.objective = M.reactions.Biomass
M.optimize()


,fluxes,reduced_costs
Biomass,0.0,-3.330669e-16
Biomass__fucose,0.0,-1.996934e+00
Biomass__phenol,0.0,-2.018078e+00
Biomass__casamino__acids,0.0,-1.990883e+00
GAM,0.0,0.000000e+00
...,...,...
norspermidine_flux,0.0,0.000000e+00
spermidine_flux,0.0,6.938894e-18
thiamin_diphosphate_flux,0.0,0.000000e+00
representative_quinone_flux,0.0,0.000000e+00


### 2 protons can be pumped out, 0.016 biomas will be produced.

In [22]:
M= model.copy()
Psi = Metabolite(
    'Psi',
    formula='',
    name='PSI is a unit of pressure expressed in pounds of force per square inch of area',
    compartment='cytosol')
M.reactions.get_by_id('T_symporter_(S)__lactic_acid').remove_from_model()
reaction = Reaction('T_symporter_(S)__lactic_acid')
reaction.name = 'T_symporter_(S)__lactic_acid'
reaction.lower_bound = -1000.  # This is the default
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"(S)__lactic_acid":-1.0,
                         "H+":-2.0,
                         "(S)__lactic_acid_ex":1.0,
                           Psi: 2.0,
                          "H+_ex":2.0})

M.reactions.get_by_id("prot_prot__lactate").bounds = (-1000,1000)

# addubg the precusors 
M.reactions.get_by_id("T_flux_oxygen").bounds = (0,0)
M.reactions.NGAM.bounds = (0,1000)
reaction = Reaction('coenzyme_B12_flux')
reaction.name = 'coenzyme_B12_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"coenzyme_B12": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('norspermidine_flux')
reaction.name = 'norspermidine_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"norspermidine": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('spermidine_flux')
reaction.name = 'spermidine_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"spermidine": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('thiamin_diphosphate_flux')
reaction.name = 'thiamin_diphosphate_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"thiamin_diphosphate": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('representative_quinone_flux')
reaction.name = 'representative_quinone_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"representative_quinone": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('norspermine_flux')
reaction.name = 'norspermine_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"norspermine": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
M.objective = M.reactions.Biomass
M.optimize()


,fluxes,reduced_costs
Biomass,0.016464,-1.054712e-15
Biomass__fucose,0.000000,-2.001281e+00
Biomass__phenol,0.000000,-1.993814e+00
Biomass__casamino__acids,0.000000,-1.997049e+00
GAM,0.016464,-1.887379e-15
...,...,...
norspermidine_flux,-0.000057,3.785167e-15
spermidine_flux,-0.000204,-1.078998e-15
thiamin_diphosphate_flux,-0.000002,-2.148844e-12
representative_quinone_flux,-0.000024,2.303019e-14


### 4 protons can be pumped out, 0.029 biomas will be produced.

In [23]:
M= model.copy()
Psi = Metabolite(
    'Psi',
    formula='',
    name='PSI is a unit of pressure expressed in pounds of force per square inch of area',
    compartment='cytosol')
M.reactions.get_by_id('T_symporter_(S)__lactic_acid').remove_from_model()
reaction = Reaction('T_symporter_(S)__lactic_acid')
reaction.name = 'T_symporter_(S)__lactic_acid'
reaction.lower_bound = -1000.  # This is the default
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"(S)__lactic_acid":-1.0,
                         "H+":-4.0,
                         "(S)__lactic_acid_ex":1.0,
                           Psi: 4.0,
                          "H+_ex":4.0})

M.reactions.get_by_id("prot_prot__lactate").bounds = (-1000,1000)

# addubg the precusors 
M.reactions.get_by_id("T_flux_oxygen").bounds = (0,0)
M.reactions.NGAM.bounds = (0,1000)
reaction = Reaction('coenzyme_B12_flux')
reaction.name = 'coenzyme_B12_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"coenzyme_B12": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('norspermidine_flux')
reaction.name = 'norspermidine_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"norspermidine": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('spermidine_flux')
reaction.name = 'spermidine_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"spermidine": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('thiamin_diphosphate_flux')
reaction.name = 'thiamin_diphosphate_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"thiamin_diphosphate": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('representative_quinone_flux')
reaction.name = 'representative_quinone_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"representative_quinone": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('norspermine_flux')
reaction.name = 'norspermine_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"norspermine": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
M.objective = M.reactions.Biomass
M.optimize()


,fluxes,reduced_costs
Biomass,0.029049,-4.996004e-16
Biomass__fucose,0.000000,-2.001569e+00
Biomass__phenol,0.000000,-1.991302e+00
Biomass__casamino__acids,0.000000,-2.002355e+00
GAM,0.029049,-7.771561e-16
...,...,...
norspermidine_flux,-0.000100,1.734723e-17
spermidine_flux,-0.000359,-3.469447e-18
thiamin_diphosphate_flux,-0.000004,-3.469447e-17
representative_quinone_flux,-0.000043,2.255141e-17


### 6 protons can be pumped out, 0.039 biomas will be produced.

In [24]:
M= model.copy()
Psi = Metabolite(
    'Psi',
    formula='',
    name='PSI is a unit of pressure expressed in pounds of force per square inch of area',
    compartment='cytosol')
M.reactions.get_by_id('T_symporter_(S)__lactic_acid').remove_from_model()
reaction = Reaction('T_symporter_(S)__lactic_acid')
reaction.name = 'T_symporter_(S)__lactic_acid'
reaction.lower_bound = -1000.  # This is the default
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"(S)__lactic_acid":-1.0,
                         "H+":-6.0,
                         "(S)__lactic_acid_ex":1.0,
                           Psi: 6.0,
                          "H+_ex":6.0})

M.reactions.get_by_id("prot_prot__lactate").bounds = (-1000,1000)

# addubg the precusors 
M.reactions.get_by_id("T_flux_oxygen").bounds = (0,0)
M.reactions.NGAM.bounds = (0,1000)
reaction = Reaction('coenzyme_B12_flux')
reaction.name = 'coenzyme_B12_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"coenzyme_B12": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('norspermidine_flux')
reaction.name = 'norspermidine_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"norspermidine": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('spermidine_flux')
reaction.name = 'spermidine_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"spermidine": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('thiamin_diphosphate_flux')
reaction.name = 'thiamin_diphosphate_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"thiamin_diphosphate": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('representative_quinone_flux')
reaction.name = 'representative_quinone_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"representative_quinone": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
reaction = Reaction('norspermine_flux')
reaction.name = 'norspermine_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"norspermine": -1.0,
                          "ADP": -1.0,
                          "ATP": 1.0,
                          "phosphate": -1.0
                         })
M.objective = M.reactions.Biomass
M.optimize()


,fluxes,reduced_costs
Biomass,0.038980,-5.551115e-16
Biomass__fucose,0.000000,-2.001796e+00
Biomass__phenol,0.000000,-1.989320e+00
Biomass__casamino__acids,0.000000,-2.006543e+00
GAM,0.038980,6.661338e-16
...,...,...
norspermidine_flux,-0.000134,2.949030e-17
spermidine_flux,-0.000482,1.561251e-17
thiamin_diphosphate_flux,-0.000006,2.775558e-17
representative_quinone_flux,-0.000057,2.775558e-17


in the presence of sulfate

## Checking whether there are electron acceptors alternative to molecular oxygen 

### Check whether nitrite is being used in the model

In [25]:
for rxn in model.metabolites:
    if "nitrite" in rxn.id:
        print (rxn.id) 
print ("'Nitrite' is not used in the model.")

'Nitrite' is not used in the model.


### Check whether NO is being used in the model

In [26]:
for rxn in model.metabolites:
    if "NO"  in rxn.id:
        print (rxn.id)  
print ("'NO' is not used in the model.")

'NO' is not used in the model.


### Check whether N2O is being used in the model

In [27]:
for rxn in model.metabolites:
    if 'N2O'  in rxn.id:
        print (rxn.id)   
print ("'N2O' is not used in the model.")

'N2O' is not used in the model.


### Check whether sulfate is being used as electron acceptor in the model to produce ATP

In [28]:
M= model.copy()
Psi = Metabolite(
    'Psi',
    formula='',
    name='PSI is a unit of pressure expressed in pounds of force per square inch of area',
    compartment='cytosol')
M.reactions.get_by_id('T_symporter_(S)__lactic_acid').remove_from_model()
reaction = Reaction('T_symporter_(S)__lactic_acid')
reaction.name = 'T_symporter_(S)__lactic_acid'
reaction.lower_bound = -1000.  # This is the default
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"(S)__lactic_acid":-1.0,
                         "H+":-0.0,
                         "(S)__lactic_acid_ex":1.0,
                           Psi: 0.0,
                          "H+_ex":0.0})

M.reactions.get_by_id("prot_prot__lactate").bounds = (-1000,1000)

M.reactions.get_by_id("T_flux_hydrogen_sulfide").bounds = (-1000,1000)
M.reactions.get_by_id("T_flux_oxygen").bounds = (0,0)

M.reactions.NGAM.bounds = (0,1000)

M.objective = M.reactions.get_by_id('NGAM')
sol = M.optimize()
print ("The pathways yield",sol.fluxes.loc['NGAM'],"ATP.")

The pathways yield 0.0 ATP.


In [29]:
M= model.copy()
Psi = Metabolite(
    'Psi',
    formula='',
    name='PSI is a unit of pressure expressed in pounds of force per square inch of area',
    compartment='cytosol')
M.reactions.get_by_id('T_symporter_(S)__lactic_acid').remove_from_model()
reaction = Reaction('T_symporter_(S)__lactic_acid')
reaction.name = 'T_symporter_(S)__lactic_acid'
reaction.lower_bound = -1000.  # This is the default
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"(S)__lactic_acid":-1.0,
                         "H+":-0.0,
                         "(S)__lactic_acid_ex":1.0,
                           Psi: 0.0,
                          "H+_ex":0.0})

M.reactions.get_by_id("prot_prot__lactate").bounds = (-1000,1000)


M.reactions.get_by_id("T_flux_hydrogen_sulfide").bounds = (-1000,1000)
M.reactions.get_by_id("T_flux_oxygen").bounds = (0,0)

M.reactions.NGAM.bounds = (0,1000)
M.objective = M.reactions.get_by_id("NGAM")
sol = M.optimize()
print ("The pathways yield", sol.fluxes.loc['NGAM'],"ATP.")

The pathways yield 0.0 ATP.


### making the reaction 2.7.7.4 use ADP as co-substrate (rather than ATP) and phosphate as product, rather than PP. 

In [30]:
M= model.copy()
Psi = Metabolite(
    'Psi',
    formula='',
    name='PSI is a unit of pressure expressed in pounds of force per square inch of area',
    compartment='cytosol')
M.reactions.get_by_id('T_symporter_(S)__lactic_acid').remove_from_model()
reaction = Reaction('T_symporter_(S)__lactic_acid')
reaction.name = 'T_symporter_(S)__lactic_acid'
reaction.lower_bound = -1000.  # This is the default
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"(S)__lactic_acid":-1.0,
                         "H+":-0.0,
                         "(S)__lactic_acid_ex":1.0,
                           Psi: 0.0,
                          "H+_ex":0.0})

M.reactions.get_by_id("prot_prot__lactate").bounds = (-1000,1000)

M.reactions.get_by_id("sulf_2.7.7.4_SULFATE__ADENYLYLTRANS__RXN").remove_from_model()
reaction = Reaction('sulf_2.7.7.4_SULFATE__ADENYLYLTRANS__RXN')
reaction.name = 'sulf_2.7.7.4_SULFATE__ADENYLYLTRANS__RXN'
reaction.lower_bound = -1000.  # This is the default
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"ADP":-1.0,
                         "sulfate":-1.0,
                         "adenosine_5'__phosphosulfate":1.0,
                          "phosphate":1.0})
M.reactions.get_by_id("T_flux_hydrogen_sulfide").bounds = (-1000,1000)
M.reactions.get_by_id("T_flux_oxygen").bounds = (0,0)

M.reactions.NGAM.bounds = (0,1000)
M.objective = M.reactions.get_by_id("NGAM")
sol = M.optimize()
print ("The pathways yield", sol.fluxes.loc['NGAM'],"ATP.")

The pathways yield 0.0 ATP.


## With adding the acetic_acid export reaction and, fermentation can produce atp

In [31]:
M = model.copy()
M.reactions.get_by_id('prot_prot__acetate').bounds = (-1000,1000)
reaction = Reaction('acetic_acid_flux')
reaction.name = 'acetic_acid_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"acetic_acid": -1.0,
                         })
reaction = Reaction('n__propanol_flux')
reaction.name = 'n__propanol_flux'
reaction.lower_bound = -1000.  # This is the default  
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"n__propanol": -1.0
                         })
M.reactions.get_by_id("T_flux_oxygen").bounds = (0,0)
M.objective = M.reactions.NGAM
sol = M.optimize()
l = sol.fluxes[abs(sol.fluxes)>1e-6].index

print ("The fluxes through NGAM reactions is", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc1)
b

The fluxes through NGAM reactions is 0.3999999999997571


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

## Adding the exist for all the metabolites and check the production of ATP, fig. S9
--biomass production as the objective reaction 
-- ask for the production of atp

In [32]:
M = model.copy()
Psi = Metabolite(
    'Psi',
    formula='',
    name='PSI is a unit of pressure expressed in pounds of force per square inch of area',
    compartment='cytosol')
M.reactions.get_by_id('T_symporter_(S)__lactic_acid').remove_from_model()
reaction = Reaction('T_symporter_(S)__lactic_acid')
reaction.name = 'T_symporter_(S)__lactic_acid'
reaction.lower_bound = -1000.  # This is the default
reaction.upper_bound = 1000.  # This is the default
M.add_reactions([reaction])
reaction.add_metabolites({"(S)__lactic_acid":-1.0,
                         "H+":-0.0,
                         "(S)__lactic_acid_ex":1.0,
                           Psi: 0.0,
                          "H+_ex":0.0})

M.reactions.get_by_id("T_flux_oxygen").bounds = (0,0)
met= [m for m in model.metabolites]
for m in met:
    reaction_id = str(m) + "_flux"
    reaction = Reaction(reaction_id)
    reaction.name = reaction_id
    reaction.lower_bound = 0.  # This is the default
    reaction.upper_bound = 1000.  # This is the default
    M.add_reactions([reaction])
    reaction.add_metabolites({ m: -1.0
                              })
# M.reactions.get_by_id("T_flux_oxygen").bounds = (0,0)
M.objective = M.reactions.NGAM
sol = M.optimize()
# l = sol.fluxes[abs(sol.fluxes)>1e-6].index
# for rxn in l:
#     print (rxn,M.reactions.get_by_id(rxn).reaction,sol.fluxes.loc[rxn])

print ("The fluxes through NGAM reactions is", sol.fluxes.loc["NGAM"])
b = show_map(sol,map_loc1)
b    

The fluxes through NGAM reactions is 0.39999999999999925


Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…